In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import svm, datasets
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import auc
#from sklearn.metrics import plot_roc_curve
from sklearn.metrics import roc_curve
from sklearn.model_selection import StratifiedKFold
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import train_test_split
from lifelines.statistics import logrank_test
from lifelines.statistics import multivariate_logrank_test
from lifelines import*
from lifelines import KaplanMeierFitter
from lifelines.statistics import logrank_test
from lifelines.statistics import multivariate_logrank_test
from lifelines.utils import median_survival_times
from lifelines import CoxPHFitter  #cox-proportional hazard ratio
import matplotlib.pyplot as plt
kmf=KaplanMeierFitter()
cph=CoxPHFitter()

In [ ]:
# Data cleaning time
time = pd.read_excel(r"C:\Data\Cox\Template for Suvival analysis all LNs.xlsx", header=None)
time2 = time.replace({0:{24167:"01-24167B17","7P":"03292019_RZ_DC PD-L1 Panel_14346-1-7 SLN(N)x9",
                                "hc-04":"12195-01-04x1","hc-08":"12195-01-08",
                                "hc-18":"12195-01-18x1","hc-22":"12195-01-22x1", "hc-24":"12195-01-24x1",
                                "hc-27":"12195-01-27x1","hc-29":"12195-01-29","hc-35":"12195-01-35x1","hc-43":"12195-01-43x1",
                                "hc-44":"12195-01-45x1","hc-47":"12195-01-47x1","hc-50":"12195-01-50x1",
                                "VPt3":"12195-03-LNNA1x4","VPt12":"12195-12-LNNa1x4","VPt15":"12195-15-LNNA1x7",
                                "VPT17":"12195-17-LNNA1x3","VPt19":"12195-19-LNNA1x7","VPt20":"12195-20-LNNA1x4",
                                "VPt21":"12195-21-LNNA1x7","VPt22":"12195-22-LNNA1x4","VPt26":"12195-26-LNNA1x8",
                                "VPt28":"12195-28-LNNA1x7","VPt29":"12195-29-LNNB1x7","VPt31":"12195-31-LNNA1x4",
                                "Pt10":"14346-1-10-LNx5","Pt13":"14346-1-13-SNL(N)x6", "Pt14":"14346-1-14-LN(N)x8",
                                "Pt16":"14346-1-16-LN(N)x6","Pt2":"14346-1-2-LN-Nx5",21:"14346-1-21 Cx6","Pt22":"14346-1-22-B2x1",
                                "Pt25":"14346-1-25-A1x1","Pt26":"14346-1-26-E1x4","Pt29":"14346-1-29-E21x2",
                                "Pt30":"14346-1-30 B1x5","5P":"14346-1-5-LN(N)x10","9P":"14346-1-9-SLN(N)x11",
                        "8403G":"8403Gx2", 23:"20190222_RZ_DC PD-L1 Panel_14346-1-23 (Block #A2)x4",
                        28:"20190222_RZ_DC PD-L1 Panel_14346-1-28 (Block #B1)x6","Pt37":"20190222_RZ_DC PD-L1 Panel_14346-1-37 C1x3",
                        "Pt36":"14346-1-36-C1x2",33:"14346-1-33 A1x9",38:"14346-1-38 A1x9",39:"14346-1-39 A1x6",
                        "VPt9":"20190222_RZ_DC PD-L1 Panel_12195-09 LNNA1x5","VPt13":"20190222_RZ_DC PD-L1 Panel_12195-13 LNNA1x5",
                        "VPt23":"20190222_RZ_DC PD-L1 Panel_12195-23 LNNC2x6","Pt32":"14346-1-32 Ax1"}})
time3 = time2.rename(columns={0:"Pt",1:"Days",2:"ifRelapse"}).drop(columns=3)

# Data: proportion of DCs found in homo and heterocluster 
data = pd.read_csv(r"C:\Data\DCs_results\DCs_dedicated_to_Cluster_TotDCs.txt")
data2 = data[["Pt","CD1a+_%h","CD1a+_%het","cDC1_%h","cDC1_%het"]]
# Merge data with Size of clusters
df = pd.merge(time3, data2, on=["Pt"], how="inner")
data_clustersize = pd.read_csv(r"C:\Data\DCs_results\SIZE_cType_Pts.txt")
data_clustersize["Pt"] = data_clustersize["Unnamed: 0"]
data_clustersize = data_clustersize.drop(columns="Unnamed: 0")
df2 = pd.merge(df, data_clustersize, on=["Pt"], how="inner").fillna(0)

# DataNew: porportion of PD-L1+DCs (out of all DCs in slide)-cDC1
df_Cluster_pv2 = pd.read_csv(r"C:\Data\DCs_results\c_Type.txt")
df_Cluster_pv2_Pt = df_Cluster_pv2[~df_Cluster_pv2["Pt"].isin(["Norm LN1x4","Norm LN2x4","Norm LN3x4","Norm LN4x4"])].drop(columns=["Unnamed: 0"])
df_Cluster_pv2_PtHet = df_Cluster_pv2_Pt[(df_Cluster_pv2_Pt.c_Type=="Het")][["Pt","CD141+PD-L1+"]].pivot_table(index=["Pt"],aggfunc=lambda x:sum(x))
New_merge = pd.merge(df_Cluster_pv2_PtHet, data, on=["Pt"])
New_merge["%PD-L1HetcDC1 out all cDC1"] = 100*New_merge["CD141+PD-L1+"]/New_merge["cDC1_ct"].round(1)
New_data=New_merge[["Pt","%PD-L1HetcDC1 out all cDC1"]].round(1)
df3 = pd.merge(df2, New_data, on=["Pt"], how="inner")

# DataNew: porportion of PD-L1+DCs (out of all DCs in slide)-CD1A
df_Cluster_pv2_PtHet1A_PDL1 = df_Cluster_pv2_Pt[(df_Cluster_pv2_Pt.c_Type=="Het")][["Pt","CD1a+PD-L1+"]].pivot_table(index=["Pt"],aggfunc=lambda x:sum(x))
df_Cluster_pv2_Pt["Total CD1a+"] = df_Cluster_pv2_Pt["CD1a+PD-L1+"]+df_Cluster_pv2_Pt["CD1a+PD-L1-"]
partial1AdATA = df_Cluster_pv2_Pt[["Pt","Total CD1a+"]].pivot_table(index=["Pt"],aggfunc = lambda x:sum(x))
partial1AdATA=partial1AdATA[(partial1AdATA["Total CD1a+"]!=0)]
New_data_1A = pd.merge(df_Cluster_pv2_PtHet1A_PDL1,partial1AdATA, on=["Pt"], how="inner")
New_data_1A["%PD-L1HetCD1A out all CD1A"] = 100*New_data_1A["CD1a+PD-L1+"]/New_data_1A["Total CD1a+"].round(1)
New_data_1A["Pt"]=New_data_1A.index
New_data_1A_2=New_data_1A[["Pt","%PD-L1HetCD1A out all CD1A"]].round(1)
New_data_1A_2.reset_index(drop=True, inplace=True)
df4 = pd.merge(df3, New_data_1A_2, on=["Pt"], how="outer")
df4.to_csv(r"C:\Data\Cox\Data.txt")

In [ ]:
#Categorical (fixed cutoff)
def ClassbyOneThreshold(dTest, cutoff):
    cut = dTest.quantile([cutOff])
    catgr, gPt =[],[]
    for i in np.arange(3, len(dTest.columns)):
        g1=dTest[(dTest.iloc[:,i]>cut.iloc[0,i-1])]
        g2=dTest[(dTest.iloc[:,i]<=cut.iloc[0,i-1])]
        g1["Class%s"%dTest.columns[i]]=1
        g2["Class%s"%dTest.columns[i]]=0
        g = pd.concat([g1["Class%s"%dTest.columns[i]], g2["Class%s"%dTest.columns[i]]])
        gPt.append(pd.concat([g1,g2]))
        catgr.append(g)
        catgr_all=pd.DataFrame(catgr).transpose()
        complete_all=pd.concat(gPt,axis=1)
        dfClass = complete_all.loc[:,~complete_all.columns.duplicated()].drop(columns={"CD1a+_%h","CD1a+_%het","cDC1_%h","cDC1_%het","CD1a-h_Size",
                                                                                             "cDC1-h_size","Het_size","%PD-L1HetcDC1 out all cDC1","%PD-L1HetCD1A out all CD1A"})
    dfClass.to_csv(r"C:\Data\Cox\ClassData.txt")
    
if __name__ == '__main__':
    ClassbyOneThreshold(df4, 0.31)


In [ ]:
# ROC curve for best cutoff
def RocForCutOff(dTest): 
    Results ={"Vars":[],"Data_cutoff":[], "ROC_AUC":[]}
    for i in np.arange(3, len(dTest.columns)-1):  #the last one has NaN
        label = dTest.iloc[:,2].values.astype(int)
        predictors = dTest.iloc[:,i].values.reshape(-1,1)
        model = LogisticRegression()
        model.fit(predictors, label)
        probs = model.predict_proba(predictors)

        fpr, tpr, thresholds = roc_curve(label, probs[:,1]) 
        roc_auc = auc(fpr, tpr)
        #plt.figure()
        plt.plot(fpr, tpr, lw=1, label='ROC/AUC (area = %0.2f)' % (roc_auc))
        plt.plot([0, 1], [0, 1], '--', color=(0.6, 0.6, 0.6), label='Luck')
        plt.xlim([-0.05, 1.05])
        plt.ylim([-0.05, 1.05])
        plt.grid()
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('ROC %s'%dTest.columns[i])
        plt.legend(loc="lower right")
        #plt.savefig('C:/Users/DrHeh/Google Drive/programming/samples/Cox/ROCAUC%s.tiff'% columnOfInterest, format='tiff', dpi=600)
        plt.show() # comment out for multiple lines plot together


        optimal_thresholds = thresholds[np.argmax(tpr-fpr)]  
        data = pd.DataFrame({"%s" %dTest.columns[i]:dTest.iloc[:,i].values, 
                            "Labels":label, "Probs":probs[:,1]}).sort_values(by=["Probs"])
        cutoff = data.loc[data[data.columns[2]] == optimal_thresholds][[data.columns[0]]].iloc[0,0]

        Results["Vars"].append(dTest.columns[i])
        Results["Data_cutoff"].append(cutoff)
        Results["ROC_AUC"].append(roc_auc)


    Res = pd.DataFrame(Results)
    Res.to_csv(r"C:\Data\Cox\ROC_CutOffAUC.txt")
if __name__ == '__main__':
    RocForCutOff(df4)

In [ ]:
# Create feature-dependent cutoffs based on ROC curve:
Res=pd.read_csv(r"C:\Data\Cox\ROC_CutOffAUC.txt")
Res=Res.drop(columns="Unnamed: 0")
def ClassbyRoc(dTest):
    catgr, gPt =[],[]
    for i in np.arange(3, len(dTest.columns)-1):
        g1=dTest[(dTest.iloc[:,i]>Res.iloc[:,1][i-3])]
        g2=dTest[(dTest.iloc[:,i]<=Res.iloc[:,1][i-3])]
        g1["Class%s"%dTest.columns[i]]=1
        g2["Class%s"%dTest.columns[i]]=0
        g = pd.concat([g1["Class%s"%dTest.columns[i]], g2["Class%s"%dTest.columns[i]]])
        gPt.append(pd.concat([g1,g2]))
        catgr.append(g)
        catgr_all=pd.DataFrame(catgr).transpose()
        complete_all=pd.concat(gPt,axis=1)
        dfClassbyROC = complete_all.loc[:,~complete_all.columns.duplicated()].drop(columns={"CD1a+_%h","CD1a+_%het","cDC1_%h","cDC1_%het","CD1a-h_Size",
                                                                                             "cDC1-h_size","Het_size","%PD-L1HetcDC1 out all cDC1","%PD-L1HetCD1A out all CD1A"})
    dfClassbyROC.to_csv(r"C:\Data\Cox\ClassbyRocData.txt")


    FrequencyCount=[]
    for i in np.arange(3, len(dTest.columns)-1):
        FrequencyCount.append(dfClassbyROC["Class%s"%dTest.columns[i]].value_counts())
        Freq=pd.DataFrame(FrequencyCount)
    Freq["From Lower cutoff"]=100*Freq[0]/(Freq[0]+Freq[1])
    Freq.to_csv(r"C:\Data\Cox\RocThresholdCutoffs.txt")
    
if __name__ == '__main__':
    ClassbyRoc(df4)

In [ ]:
dfClassbyROC=pd.read_csv(r"C:\Data\Cox\ClassbyRocData.txt")
dfClassbyROC=dfClassbyROC.drop(columns="Unnamed: 0")
# ROC for all predictors (can also put multiple, most powerful ones)
def RocAllPredictor(dTest):
    predictors = dTest.iloc[:,np.r_[3:len(dTest.columns)-1]].values
    label = dTest.iloc[:,2].values.astype(int)

    model = LogisticRegression()
    model.fit(predictors, label)
    probs = model.predict_proba(predictors)

    fpr, tpr, thresholds = roc_curve(label, probs[:,1]) 
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, lw=1, label='ROC/AUC (area = %0.2f)' % (roc_auc))
    plt.plot([0, 1], [0, 1], '--', color=(0.6, 0.6, 0.6), label='Luck')
    plt.xlim([-0.05, 1.05])
    plt.ylim([-0.05, 1.05])
    plt.grid()

    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC_all predictors')
    plt.legend(loc="lower right")
    plt.savefig(r"C:\Data\Cox\allPredictorsROCAUC.pdf", format="pdf")
    plt.show()
    


# Cox Hazard Ratio- Mutivariate
def Cox(df):
    df = df.drop(columns="Pt")
    c=cph.fit(df, duration_col="Days", event_col="ifRelapse")
    bx=c.plot(hazard_ratios=True)
    

# Cox Hazard Ratio- CI of every feature 
def CoxIndvidualFeature(dTest):
    for i in np.arange(3, len(dTest.columns)-1): #the last one columns has NaN
        columnOfInterest = dTest.columns[i]
        dataOfInterest = dTest[["Days","ifRelapse",columnOfInterest]]
        c=cph.fit(dataOfInterest, duration_col="Days", event_col="ifRelapse")
        c.print_summary()
        plt.figure()
        bx=c.plot(hazard_ratios=True)
        bx.get_figure().tight_layout()
        bx.get_figure().savefig('C:/Data/Cox/%s.tiff'% columnOfInterest, format='tiff', dpi=600)
        


# KM plot for every feature
def KMplot(dTest):
    for i in np.arange(3, len(dTest.columns)):
        g_high = dTest[(dTest.iloc[:,i]==1)]
        g_low = dTest[(dTest.iloc[:,i]==0)]

        plt.figure()
        print("KM curves")
        kmf1=kmf.fit(g_high["Days"], g_high["ifRelapse"], label="Hi %s"%dTest.columns[i])
        ax=kmf.plot(color="red", ci_show=False,show_censors=True)

        kmf1=kmf.fit(g_low["Days"], g_low["ifRelapse"], label="Low")
        ax=kmf.plot(color="black", ci_show=False,show_censors=True)
        plt.ylim(0,1.1)

        ax.get_figure().savefig("C:/Data/Cox/KM%s.png"%dTest.columns[i])
        

# Logrank for every features (once calssified):
def logrank(dTest):
    Logp, Vars  =[],[]
    for i in np.arange(3, len(dTest.columns)):
        g_high = dTest[(dTest.iloc[:,i]==1)]
        g_low = dTest[(dTest.iloc[:,i]==0)]
        lr = logrank_test(g_high.Days, g_low.Days, g_high.ifRelapse, g_low.ifRelapse)
        Vars.append(dTest.columns[i])
        Logp.append(lr.p_value)
    Log=pd.DataFrame({"logp":Logp, "Vars":Vars})
    Log.to_csv("C:/Data/Cox/Logrankp.txt")

if __name__ == '__main__':
    RocAllPredictor(dfClassbyROC)
    Cox(dfClassbyROC)
    CoxIndvidualFeature(dfClassbyROC)
    KMplot(dfClassbyROC)
    logrank(dfClassbyROC)